In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import requests
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from bs4 import BeautifulSoup

In [3]:
URL = 'https://www.worldometers.info/coronavirus/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
countries_table =soup.find_all(id='main_table_countries_today')
df = pd.read_html(str(countries_table),index_col=False)[0]

In [4]:
df.sort_values(by=['TotalCases'],ascending=False,inplace=True)

In [5]:
df_top_50=df.iloc[2:52].fillna(0).copy(deep=False)
df_top_50['covid_rank'] = range(1, 1+len(df_top_50))

In [6]:
fig_bar = go.Figure()
fig_bar.add_trace(go.Bar(
    x=df_top_50['Country,Other'],
    y=df_top_50['TotalCases'],
    name='Total Case',
 
    marker=dict(
        color='rgba(246, 78, 139, 0.6)'
    )
))
fig_bar.add_trace(go.Bar(
    x=df_top_50['Country,Other'],
    y=df_top_50['TotalRecovered'],
    name='Total Recovered',
 
    marker=dict(
        color='rgba(58, 71, 80, 0.6)'
    )
))


fig_bar.add_trace(go.Bar(
    x=df_top_50['Country,Other'],
    y=df_top_50['TotalDeaths'],
    name='Total Death',
 
    marker=dict(
        color='rgba(231, 71, 80, 0.6)'
    )
))
fig_bar.update_layout( title = {"text": "<span style='font-size:2em;color:black'>COVID-19 CORONAVIRUS PANDEMIC </span>"},
                   barmode='overlay')
fig_bar.show()

In [7]:
px.scatter(
     df_top_50,     
     y=df_top_50['TotalCases'],
     x=df_top_50['TotalDeaths'],
     color=df_top_50['Country,Other'],
     size =df_top_50['TotalCases'],
     size_max =100)

In [8]:
figtable = go.Figure(data=[go.Table(header=dict(values=['Rank','Country', 'Total Case','Total Deaths' ,'New Cases']),
                 cells=dict(values=[df_top_50['covid_rank'],df_top_50['Country,Other'], df_top_50['TotalCases'],df_top_50['TotalDeaths'],df_top_50['NewCases'].fillna(0)]))
                     ])
figtable.show()

In [11]:
nepal_data = df[df['Country,Other'] =='Nepal'].fillna(0)
nep_total_case = int(nepal_data['TotalCases'])
nep_recovered_case = nep_total_case - int(nepal_data['TotalRecovered'])
nep_newcase = int(nepal_data['NewCases'])

In [12]:
fig = go.Figure(go.Indicator(
    mode = "number+delta",
    value = nep_total_case,
    delta = {'position': "top", 'reference': nep_recovered_case},
    title = {"text": "Total Case<br><span style='font-size:0.8em;color:gray'>Recovered</span>"},
    domain = {'row': 0, 'column': 0}))
fig.add_trace(go.Indicator(
    mode = "number",
    value = nep_newcase,
    title = {"text": "<span style='font-size:0.8em;color:red'>New case </span>"},
    domain = {'row': 0, 'column': 1}))

fig.update_layout( title = {"text": "<span style='font-size:2em;color:black'>COVID-19 CASE IN NEPAL</span>"},
    grid = {'rows': 1, 'columns': 2, 'pattern': "independent"})
fig.show()